# FEM code for linear elasticity

In [ ]:
infile = open("mesh_1.inp", "r")
outfile = open("result_1.out", "w")

## Input FEM data

This function reads the input file for the FEM analysis of two dimensional elasticty using three, and eight node isoparameteric elements:

Variable and array list:
--------------------------------
    npoin: Total number of nodes in the solution
    nelem: Total number of elements
    nvfix: Total number of constrained nodes
    ntype: Solution type(ntype  = 1, plane-stress and ntypes = 2 plain-strain).
    nnode: Number of nodes per element.
    ndofn: Number of degree of freedom (DOF) per node
    ndime: Number of spatial dimensions.
    ngaus: The order of numerical intergration.
    nmats: Total number of different materials in the solution.
    nstre: Number of stress components.
    nprop: Number of material properties.
    matno(nelem): Material types for the elements.
    nofix(nvfix): Node numbers at which one or more DOFs are constrained.
    lnods(nelem,nnode): Element nodal connectivity list.
    coord(npoin,ndime): Cartesian coordinates of each node.
    iffix(nvfix,ndofn): List of constrained DOFs.
    fixed(nvfix,ndofn): Prescribed value of any constrained DOFs.
    props(nmats,nprops): For each different material, the properties of that material.
    
    

In [ ]:
function input_fem_elast(infile,out)
    header = split(chomp(readline(infile)))
    npoin, nelem, nvfix, ntype, nnode, ndofn, ndime, ngaus, nstre, nmats, nprop = parse.(Int, header)
    lnods = zeros(Int64,nelem,nnode)
    matno = zeros(Int64,nelem)
    coord = zeros(npoin,ndime)
    nofix = zeros(Int64,nvfix)
    iffix = zeros(Int64,nvfix, ndofn)
    fixed = zeros(nvfix, ndofn)
    props = zeros(nmats, nprop)
    jelem = 0
    
    for ielem = 1:nelem
        eleminf = split(chomp(readline(infile)))
        data = parse.(Int, eleminf)
        jelem = data[1]
        lnods[jelem,:] = data[2:nnode+1]
        matno[jelem] = data[nnode+2]
    end
        
    # Nodal coordinates
    for ipoin = 1:npoin
        point = split(chomp(readline(infile)))
        jpoin = parse(Int, point[1])
        coord[ipoin,:] = parse.(Float64,point[2:3])
    end
    # Constraint nodes and their values:
    for ivfix = 1:nvfix
        fix = split(chomp(readline(infile)))
        nofix[ivfix] = parse(Int,fix[1])
        dummy1 = parse.(Int, fix[2:3])
        dummy2 = parse.(Float64, fix[3:4])
        for idofn = 1 : ndofn
            iffix[ivfix,idofn] = dummy1[idofn]
            fixed[ivfix,idofn] = dummy2[idofn]
        end
    end
    
    # material properties of the system
    for imats = 1:nmats
        mat = split(chomp(readline(infile)))
        jmats = parse(Int, mat[1])
        props[jmats,1:2] .= parse.(Float64, mat[3:end])
    end
    # this is because the file is not read after this point
    
    println(out,"***********************")
    println(out,"*   FEM input data    *")
    println(out,"***********************")
    println(out,"Number of Elements          : ", nelem)
    println(out,"Number of Nodes             : ", npoin)
    println(out,"Number of fixed nodes       : ", nvfix)
    println(out,"Number of nodes per elem    : ", nnode)
    println(out,"Number of integration points: ", ngaus)
    println(out,"Number of materials         : ", nmats)
    println(out,"Number of properties        : ", nprop)
    
    return  (npoin, nelem, nvfix, ntype, nnode, ndofn, ndime, ngaus, nstre, nmats, nprop, lnods, matno,coord,props,
            nofix, iffix, fixed) 
end    

This function evaluates the position of samplnig points and associated weights for chosen order of numerical integration.

In [ ]:
function gauss(ngaus, nnode)
    posgp = zeros(2*ngaus)
    weigp = zeros(ngaus)
    
    if (nnode == 3)
        if (ngaus == 1)
            posgp[1] = 1.0/3.0
            posgp[2] = 1.0/3.0
            weigp[1] = 0.5
        end
        
        if (ngaus == 3)
            posgp[1] = 0.5
            posgp[2] = 0.5
            posgp[3] = 0.0
            
            posgp[4] = 0.0
            posgp[5] = 0.5
            posgp[6] = 0.5
            
            weigp[1] = 1.0/6.0
            weigp[2] = 1.0/6.0
            weigp[3] = 1.0/6.0    
        end
        
        if (ngaus == 7)
            posgp[1] = 0.0
            posgp[2] = 0.5
            posgp[3] = 1.0
            posgp[4] = 0.5
            posgp[5] = 0.0
            posgp[6] = 0.0
            posgp[7] = 1.0/3.0
            
            posgp[8] = 0.0
            posgp[9] = 0.0
            posgp[10] = 0.0
            posgp[11] = 0.5
            posgp[12] = 1.0
            posgp[13] = 0.5
            posgp[14] = 1.0/3.0
            
            weigp[1] = 1.0/40.0
            weigp[2] = 1.0/15.0
            weigp[3] = 1.0/40.0
            weigp[4] = 1.0/15.0
            weigp[5] = 1.0/40.0
            weigp[6] = 1.0/15.0
            weigp[7] = 9.0/40.0
            
        end
    end
        
    if (nnode != 3)
        if(ngaus == 2)
            posgp[1] = -0.57735026918963
            weigp[1] = 1.0
        end
        if(ngaus > 2)
            posgp[1] = -0.7745966241483
            posgp[2] = 0.0
            weigp[1] = 0.55555555555556
            weigp[2] = 0.88888888888889
        end
        kgaus =  Int(floor(ngaus/2))
        print(kgaus)
        for igash = 1:kgaus
            jgash = ngaus+1-igash
            posgp[jgash] = -posgp[igash]
            weigp[jgash] = weigp[igash]
        end
    end   
    return posgp, weigp
end


In [ ]:
npoin, nelem, nvfix, ntype, nnode, ndofn, ndime, ngaus, nstre, nmats, nprop, lnods, matno, coord, props, nofix, iffix, fixed = input_fem_elast(infile,outfile)
ntotc = npoin * ndofn
posgp, weigp = gauss(ngaus, nnode)
# Form global stifness matrix (lhs)
using NBInclude
@nbinclude("stiffness.ipynb")
gstif = stiffness(npoin, nelem, nnode, nstre, ndime, ndofn, 
    ngaus, ntype, lnods, matno, coord, props, posgp, weigp)

This function integrates the prescribed distributed external loads at the element edges
and also the prescribed point loads at the nodes into the global force vector. 
Variable and array list:
--------------------------------

    npoin: Total number of nodes in the solution
    nelem: Total number of element in the solution
    ndofn: Number of DOFs per node.
    ngaus: The order of numerical integration.
    ndime: Number of global Cartesian coordinates.
    eload(nelem,nevab): Element force vector(nevab = nnode * ndofn)
    

In [ ]:
using Printf
function loads(infile, out, npoin,nelem,ndofn,nnode,ngaus,ndime,posgp,weigp,lnods,coord)
    # Initialize global force vector & elements loads
    nevab = nnode*ndofn
    ntotv = npoin*ndofn
    
    for itotv = 1:ntotv
        gforce = zeros(itotv,1)
    end
    eload = zeros(nelem,nevab)
    for ielem = 1:nelem
        for ievab = 1:nevab
            eload[ielem,ievab] = 0.0
        end
    end
    
    

    # loading types
    iplod, nedge = parse.(Int, split(chomp(readline(infile))))
    println(iplod, nedge )    
    # point forces:
    if (iplod != 0)
        pointdat = split(chomp(readline(infile)))
        println(pointdat)
        nplod = parse(Int, pointdat[1])
        lodpt = parse(Int,pointdat[2])
        dummy = parse.(Int,pointdat[3:4])
        point = zeros(ndofn)
        for idofn = 1:ndofn
            point[idofn] = dummy[idofn]
        end
        
        for ielem = 1:nelem
            for inode = 1:nnode
                nloca = lnods[ielem,inode]
                if (lodpt == nloca)
                    for idofn = 1:ndofn
                        nposi = (inode - 1)*ndofn + idofn
                        eload[ielem,nposi] = point[idofn]
                    end
                end
            end
        end
    end
    
    # Distributed forces
    if (nedge != 0)
        @printf(out, "Number of loaded edges: %5d\n",nedge)
        @printf(out, "List of loaded edges and applied loads:\n")
        for iedge = 1 : nedge
            nodeg = 3
            if (nnode != 8)
                nodeg = 2
            end
            press = zeros(nodeg,ndofn)
            # Reload loads
            loadsdat = split(chomp(readline(infile)))
            neass = parse(Int,loadsdat[1])
            noprs = parse.(Int,loadsdat[2:1+nodeg])
            
            for iodeg = 1 : nodeg
                dummy = split(chomp(readline(infile)))
                press[iodeg,:] = parse.(Float64, dummy)
            end
            
            # Print
            @printf(out,"\n")
            @printf(out,"%5d", neass)
            for iodeg = 1:nodeg
                @printf(out, "%5d", noprs[iodeg])
            end
            
            @printf(out,"\n")
            for iodeg = 1:nodeg
                for idofn = 1:ndofn
                    @printf(out,"%14.6e", press[iodeg,idofn])
                end
            end
            @printf(out,"\n")
            
            # end of reading
            
            #--Integrate along the edges
            etasp = -1.0
            
            elcod = zeros(ndime,nodeg)
            
            for iodeg = 1:nodeg
                lnode = noprs[iodeg]
                for idime = 1:ndime
                    elcod[idime,iodeg] = coord[lnode,idime]
                end
            end
            
            pgash = zeros(ndofn)
            dgash = zeros(ndofn)
            
            for igaus = 1:ngaus
                exisp = posgp[igaus]
                shape, deriv = sfr2(exisp,etasp,nnode)
                println(deriv)
                for idofn = 1 : ndofn
                    pgash[idofn] = 0.0
                    dgash[idofn] = 0.0
                    for iodeg = 1:nodeg
                        pgash[idofn] = pgash[idofn]+press[iodeg,idofn]*shape[iodeg]
                        dgash[idofn] = dgash[idofn]+elcod[idofn,iodeg]*deriv[1,iodeg]
                    end
                end
                
                dvolu = weigp[igaus]
                pxcom = dgash[1]*pgash[2]-dgash[2]*pgash[1]
                pycom = dgash[1]*pgash[1]+dgash[2]*pgash[2]
                
                for inode = 1:nnode
                    nloca = lnods[neass,inode]
                    if(nloca == noprs[1])
                        jnode = inode + nodeg - 1
                        kount = 0
                        for knode = inode : jnode
                            kount = kount + 1
                            ngash = (knode - 1) *ndofn + 1
                            mgash = (knode - 1) *ndofn + 2
                            if (knode > nnode)
                                ngash = 1
                                mgash = 2
                            end
                            eload[neass,ngash] = eload[neass,ngash]+pxcom*dvolu*shape[kount]
                            eload[neass,mgash] = eload[neass,mgash]+pycom*dvolu*shape[kount]
                        end
                    end
                end
            end
        end
    end
    
    
    # print nodal forces
    @printf(out,"\n")
    @printf(out,"Nodal forces for elements:\n")
    for ielem = 1:nelem
        @printf(out, "Element No: %d\n",ielem)
        for ievab = 1:nevab
            @printf(out, "%14.6e", eload[ielem,ievab])
            if ((nnode==8) && (ievab == nevab/2))
                 @printf(out,"\n")
            end
        end
        @printf(out,"\n")
    end
    
    gforce = zeros(ntotv)
    # Generate global force vector
    for ielem =  1 : nelem
        for inode = 1 : nnode
            lnode = lnods[ielem,inode]
            for idofn = 1:ndofn
                itotv = (lnode-1)*ndofn+idofn
                ievab = (inode-1)*ndofn+idofn
                gforce[itotv] = gforce[itotv]+eload[ielem,ievab]
            end
        end
    end
    return gforce
end

This function rearranges the stiffness matrix and force vector for prescribed boundary conditions.

In [ ]:
function boundary_cond(npoin,nvfix,nofix,iffix,fixed,ndofn,gstif,gforce)
    ntotv = npoin*ndofn
    for ivfix = 1 : nvfix
        lnode = nofix[ivfix]
        
        for idofn = 1 : ndofn
            if(iffix[ivfix,idofn] == 1)
                itotv = (lnode - 1)*ndofn+idofn
                for jtotv = 1:ntotv
                    gstif[itotv,jtotv] = 0.0
                end
                
                gstif[itotv,itotv] = 1.0
                gforce[itotv] = fixed[ivfix,idofn]
            end
        end
    end
    return gstif, gforce
end

## Force vector and Boundary Conditions

In [ ]:
gforce = loads(infile, outfile, npoin, nelem, ndofn, nnode, ngaus, ndime, posgp, weigp, lnods, coord)
gstif, gforce = boundary_cond(npoin, nvfix, nofix, iffix, fixed, ndofn, gstif, gforce)

## Solve displacements

In [ ]:
asdis = gstif\gforce

This function calculates the strain and stress values at the integration points of the elements resulting from the displacements.

In [ ]:
function stress(asdis, nelem, npoin, nnode, ngaus,nstre,props,ntype,ndofn,ndime,lnods,matno,coord,posgp,weigp)
    ngaus2 = ngaus
    
    if(nnode == 3)
        ngaus2 = 1
    end
    
    mgaus = ngaus * ngaus2
    nevab = nnode*ndofn
    
    elem_stres = zeros(nelem, mgaus, nstre+1)
    
    for ielem = 1 : nelem
        
        # Material parameters and elasticity matrix
        mtype = matno[ielem]
        dmatx = modps(mtype,ntype,nstre,props)
        poiss = props[mtype,2]
        
        eldis = zeros(ndofn, nnode)
        elcod = zeros(ndofn, nnode)
        # Nodal displacements
        for inode = 1:nnode
            lnode = lnods[ielem,inode]
            for idofn = 1:ndofn
                nposn = (lnode - 1)*ndofn + idofn
                eldis[idofn,inode] = asdis[nposn]
                elcod[idofn,inode] = coord[lnode,idofn]
            end
        end
        
        # Integrate stresses
        kgasp = 0
        for igaus = 1:ngaus
            for jgaus = 1:ngaus
                
                kgasp = kgasp + 1
                exisp = posgp[igaus]
                etasp = posgp[jgaus]
                
                if (ngaus == 1)
                    etasp = posgp[ngaus+igaus]
                end
                
                shape, deriv = sfr2(exisp, etasp, nnode)
                
                cartd, djacb, gpcod = jacob2(ielem, elcod, kgasp, shape, deriv, nnode, ndime)
                
                bmatx = bmats(nevab,nstre,cartd, shape, nnode)
                
                stran = zeros(nstre)
                # Calculate the strains
                for istre = 1:nstre
                    stran[istre] = 0.0
                    for inode = 1 : nnode
                        for idofn = 1 : ndofn
                            ievab = (inode-1)*ndofn+idofn
                            stran[istre] = stran[istre] + bmatx[istre,ievab] * eldis[idofn,inode]
                        end
                    end
                end
                
                stres = zeros(nstre+1)
                # Calculate stresses
                for istre = 1:nstre
                    stres[istre] = 0.0
                    for jstre = 1:nstre
                        stres[istre] = stres[istre]+dmatx[istre,jstre]*stran[jstre]
                    end
                end
                
                if (ntype == 1)
                    stres[4] = 0.0
                end
                if (ntype == 2)
                    stres[4] = poiss*(stres[1] + stres[2])
                end
                
                for istre = 1 : nstre + 1
                    elem_stres[ielem,kgasp,istre] = stres[istre]
                end
                
            end
        end
    end
    return elem_stres  
end    

## Calculate stresses

In [ ]:
elem_stress = stress(asdis, nelem, npoin, nnode, ngaus, nstre, props, ntype, 
 ndofn, ndime, lnods, matno, coord, posgp, weigp)

## Output results

This function prints out the solution results to file in tabulated form and in vtk file format for viewing results by using Paraview.     

In [ ]:
using NBInclude
@nbinclude("output_fem.ipynb")
output_fem(outfile, npoin, nelem, nnode, lnods, coord, ndofn, ngaus, nstre, asdis, elem_stress)
print("Done")
close(outfile) # this is necessary to flush the file